## Day 2 exercise - Three way conversation between LLMS
We will be asking GPT-4o-mini, Claude Haiku and Llama 3.2 to collectively write the lyrics for a song.

I created a Conversation class to help with reusability (also that way it's easier to transfer and run in a standalone python environment).

This generally works quite well, although Claude Haiku can be quite chatty, providing explanations (or several lines of lyrics...) despite the instructions. 

Enjoy, 

Lefteris

In [ ]:
# Let's make a conversation between GPT-4o-mini, Claude-3-haiku and Llama 3.2 (through Ollama)
# We're using cheap versions of models so the costs will be minimal
# Each model will contribute one song lyric line in turn to the conversation
# Turns: 1. GPT, 2. Claude, 3. Llama
# This code is designed to run in a Python environment with the necessary libraries installed.

# Conversation class 

class Conversation:
    def __init__(self):
        # Environment variables and model names
        from dotenv import load_dotenv
        import os
        load_dotenv()
        self.gpt_model = os.getenv("GPT_MODEL", "gpt-4o-mini")
        self.claude_model = os.getenv("CLAUDE_MODEL", "claude-3-haiku-20240307")
        self.OLLAMA_MODEL = os.getenv("OLLAMA_MODEL", "llama3.2")
        self.OLLAMA_API = "http://localhost:11434/v1"
        self.gpt_system = "You are a songwriter chatbot. You will find the next song lyric line that continues the song. Respond only with a single lyric line, without any preamble or explanation."
        self.claude_system = "You are a songwriter chatbot. You will find the next song lyric line that continues the song. Respond only with a single lyric line, without any preamble or explanation."
        self.ollama_system = "You are a songwriter chatbot. You will find the next song lyric line that continues the song. Respond only with a single lyric line, without any preamble or explanation."

        # Initialize message histories
        self.gpt_messages = ["In the still of the night\n"]
        self.claude_messages = ["I held you\n"]
        self.ollama_messages = ["Held you tight\n"]

        # Initialize API clients (replace with actual client initialization)
        import openai
        import anthropic
        from openai import OpenAI
        import ollama

        self.openai = OpenAI()
        self.claude = anthropic.Anthropic()
        self.ollama_via_openai = OpenAI(base_url=self.OLLAMA_API, api_key='ollama')

        print(f"{self.gpt_model}:\n{self.gpt_messages[0]}\n")
        print(f"{self.claude_model}:\n{self.claude_messages[0]}\n")
        print(f"{self.OLLAMA_MODEL}\n{self.ollama_messages[0]}")

    def call_gpt(self):
        messages = [{"role": "system", "content": self.gpt_system}]
        for gpt, claude, ollama in zip(self.gpt_messages, self.claude_messages, self.ollama_messages):
            messages.append({"role": "assistant", "content": gpt})
            messages.append({"role": "user", "content": f"{claude}\n{ollama}"})
        completion = self.openai.chat.completions.create(
            model=self.gpt_model,
            messages=messages
        )
        return completion.choices[0].message.content

    def call_claude(self):
        messages = []
        for gpt, claude_message, ollama in zip(self.gpt_messages, self.claude_messages, self.ollama_messages):
            messages.append({"role": "user", "content": gpt})
            messages.append({"role": "assistant", "content": claude_message})
            messages.append({"role": "user", "content": ollama})
        messages.append({"role": "user", "content": self.gpt_messages[-1]})
        message = self.claude.messages.create(
            model=self.claude_model,
            system=self.claude_system,
            messages=messages,
            max_tokens=500
        )
        return message.content[0].text

    def call_ollama(self):
        messages = [{"role": "system", "content": self.ollama_system}]
        for gpt, claude, ollama in zip(self.gpt_messages, self.claude_messages, self.ollama_messages):
            messages.append({"role": "assistant", "content": ollama})
            messages.append({"role": "user", "content": f"{gpt}\n{claude}"})
        last_msg = f"{self.gpt_messages[-1]}\n{self.claude_messages[-1]}"
        messages.append({"role": "user", "content": last_msg})
        completion = self.ollama_via_openai.chat.completions.create(
            model=self.OLLAMA_MODEL,
            messages=messages
        )
        return completion.choices[0].message.content

    def step(self):
        gpt_next = self.call_gpt()
        print(f"{self.gpt_model}:\n{gpt_next}\n")
        self.gpt_messages.append(gpt_next)

        claude_next = self.call_claude()
        print(f"{self.claude_model}\n{claude_next}\n")
        self.claude_messages.append(claude_next)

        ollama_next = self.call_ollama()
        print(f"{self.OLLAMA_MODEL}:\n{ollama_next}\n")
        self.ollama_messages.append(ollama_next)

In [ ]:
# Now we initialize the class and step through the conversation.
convo = Conversation()
for _ in range(5):
    convo.step()
